In [1]:
import openml
import os
import numpy as np
# import train test split
from sklearn.model_selection import train_test_split
openml.config.cache_directory = os.path.expanduser('/storage/store/work/lgrinszt/openml_cache')
task_id = 361071
task = openml.tasks.get_task(task_id)
X, y = task.get_X_and_y()
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Restrict X_train to 1000 samples to speed up the example
X_train, y_train = X_train[:1000], y_train[:1000]

(57580, 54) (57580,)


In [2]:
print(2)

2


In [ ]:

import random
import time
import warnings
from datetime import datetime

import torch

import numpy as np

import matplotlib.pyplot as plt
from scripts.differentiable_pfn_evaluation import eval_model_range
from scripts.model_builder import get_model, get_default_spec, save_model, load_model
from scripts.transformer_prediction_interface import transformer_predict, get_params_from_config, load_model_workflow

from scripts.model_configs import *

from datasets import load_openml_list, open_cc_dids, open_cc_valid_dids
from priors.utils import plot_prior, plot_features
from priors.utils import uniform_int_sampler_f

from scripts.tabular_metrics import calculate_score_per_method, calculate_score
from scripts.tabular_evaluation import evaluate

from priors.differentiable_prior import DifferentiableHyperparameterList, draw_random_style, merge_style_with_info
from scripts import tabular_metrics
from notebook_utils import *
import argparse
import wandb 
large_datasets = True
max_samples = 10000 if large_datasets else 5000
bptt = 10000 if large_datasets else 3000
suite='cc'
base_path = '.'
max_features = 100

args = argparse.Namespace()
args.prior = "trees"
args.task_type = 'multiclass'
args.device = 4
args.wandb = False
args.name = "test"
args.save_every = 59

device = 'cuda:{}'.format(args.device) if args.device >= 0 else 'cpu'


def print_models(model_string):
    print(model_string)

    for i in range(80):
        for e in range(50):
            exists = Path(os.path.join(base_path, f'models_diff/prior_diff_real_checkpoint{model_string}_n_{i}_epoch_{e}.cpkt')).is_file()
            if exists:
                print(os.path.join(base_path, f'models_diff/prior_diff_real_checkpoint{model_string}_n_{i}_epoch_{e}.cpkt'))
        print()
def train_function(config_sample, i, add_name=''):
    start_time = time.time()
    N_epochs_to_save = 50
    
    def save_callback(model, epoch):
        if not hasattr(model, 'last_saved_epoch'):
            model.last_saved_epoch = 0
        if ((time.time() - start_time) / (maximum_runtime * 60 / N_epochs_to_save)) > model.last_saved_epoch:
            print('Saving model..')
            config_sample['epoch_in_training'] = epoch
            save_model(model, base_path, f'models_diff/prior_diff_real_checkpoint{add_name}_n_{i}_epoch_{model.last_saved_epoch}.cpkt',
                           config_sample)
            model.last_saved_epoch = model.last_saved_epoch + 1 # TODO: Rename to checkpoint
    
    model = get_model(config_sample
                      , device
                      , should_train=True
                      , verbose=1
                      , epoch_callback = save_callback)
    
    return



def reload_config(config_type='causal', task_type='multiclass', longer=0):
    config = get_prior_config(config_type=config_type)
    
    config['prior_type'], config['differentiable'], config['flexible'] = args.prior, True, True
    
    model_string = ''
    
    config['epochs'] = 12000
    config['recompute_attn'] = True

    config['max_num_classes'] = 10
    config['num_classes'] = uniform_int_sampler_f(2, config['max_num_classes'])
    config['balanced'] = False
    model_string = model_string + '_multiclass'
    
    model_string = model_string + '_'+datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    
    return config, model_string

config, model_string = reload_config(longer=1)

config['bptt_extra_samples'] = None

config["sampling"] = "mixed"
del config['differentiable_hyperparameters']['sampling']
config["num_classes"] = 2

if config['prior_type'] == 'trees':
    config["n_trees"] = 25
    config["max_depth"] = 10
    config["depth_distribution"] = "uniform"
    config["split_distribution"] = "uniform"
    config["split_param"] = 1
elif config['prior_type'] == 'mlp':
    config['output_multiclass_ordered_p'] = 0.
    del config['differentiable_hyperparameters']['output_multiclass_ordered_p']

    config['multiclass_type'] = 'rank'
    del config['differentiable_hyperparameters']['multiclass_type']

    config['pre_sample_causes'] = True



config['multiclass_loss_type'] = 'nono' # 'compatible'
config['normalize_to_ranking'] = False # False

config['categorical_feature_p'] = .2 # diff: .0

# turn this back on in a random search!?
config['nan_prob_no_reason'] = .0
config['nan_prob_unknown_reason'] = .0 # diff: .0
config['set_value_to_nan'] = .1 # diff: 1.

config['normalize_with_sqrt'] = False

config['new_mlp_per_example'] = True
config['prior_mlp_scale_weights_sqrt'] = True
config['batch_size_per_gp_sample'] = None

config['normalize_ignore_label_too'] = False

config['differentiable_hps_as_style'] = False
config['max_eval_pos'] = 1000

config['random_feature_rotation'] = True
config['rotate_normalized_labels'] = True

config["mix_activations"] = False # False heisst eig True

config['emsize'] = 512
config['nhead'] = config['emsize'] // 128
config['bptt'] = 1024+128
config['canonical_y_encoder'] = False

    
config['aggregate_k_gradients'] = 8
config['batch_size'] = 8*config['aggregate_k_gradients']
config['num_steps'] = 1024//config['aggregate_k_gradients']
config['epochs'] = 400
config['total_available_time_in_s'] = None #60*60*22 # 22 hours for some safety...

config['train_mixed_precision'] = True
config['efficient_eval_masking'] = True

config["use_wandb"] = args.wandb
config["name"] = args.name
config["save_every"] = args.save_every

if args.wandb == True:
    wandb.init(project="tabpfn_training", entity="leogrin")
    wandb.config.update(config)

config_sample = evaluate_hypers(config)

/home/soda/lgrinszt/.local/miniconda3/envs/tab_pfn2/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [ ]:
def load_model(path, filename, device, config_sample, verbose=0):
    # TODO: This function only restores evaluation functionality but training canät be continued. It is also not flexible.
    # print('Loading....')
    model_state = torch.load(
        os.path.join(path, filename), map_location='cpu')
    if ('differentiable_hyperparameters' in config_sample
            and 'prior_mlp_activations' in config_sample['differentiable_hyperparameters']):
        config_sample['differentiable_hyperparameters']['prior_mlp_activations']['choice_values_used'] = config_sample[
                                                                                                         'differentiable_hyperparameters'][
                                                                                                         'prior_mlp_activations'][
                                                                                                         'choice_values']
        config_sample['differentiable_hyperparameters']['prior_mlp_activations']['choice_values'] = [
            torch.nn.Tanh for k in config_sample['differentiable_hyperparameters']['prior_mlp_activations']['choice_values']]

    config_sample['categorical_features_sampler'] = lambda: lambda x: ([], [], [])
    config_sample['num_features_used_in_training'] = config_sample['num_features_used']
    config_sample['num_features_used'] = lambda: config_sample['num_features']
    config_sample['num_classes_in_training'] = config_sample['num_classes']
    config_sample['num_classes'] = 2
    config_sample['batch_size_in_training'] = config_sample['batch_size']
    config_sample['batch_size'] = 1
    config_sample['bptt_in_training'] = config_sample['bptt']
    config_sample['bptt'] = 10
    config_sample['bptt_extra_samples_in_training'] = config_sample['bptt_extra_samples']
    config_sample['bptt_extra_samples'] = None

    #print('Memory', str(get_gpu_memory()))

    model = get_model(config_sample, device=device, should_train=False, verbose=verbose)
    module_prefix = 'module.'
    model_state = {k.replace(module_prefix, ''): v for k, v in model_state.items()}
    model[2].load_state_dict(model_state)
    model[2].to(device)
    model[2].eval()

    return model, config_sample

In [5]:
#model_pytorch = load_model("tabpfn/model_checkpoints", "model_mlp60400_140.pt", 2, config_sample, 1)[0]
model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees25524_160.pt", 2, config_sample, 1)[0]

Using style prior: True
{'recompute_attn': True}
<module 'wandb' (namespace)>
Using 2 device
Batch size: 1
Using distributed training: False
Using a Transformer with 25.82 M parameters


In [6]:
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier

model = TabPFNClassifier()

Using style prior: True
{'recompute_attn': True}
<module 'wandb' (namespace)>
Using cpu device
Batch size: 1
Using distributed training: False
Using a Transformer with 25.82 M parameters


In [17]:
model.c

{'lr': 0.0001,
 'dropout': 0.0,
 'emsize': 512,
 'batch_size': 1,
 'nlayers': 12,
 'num_features': 100,
 'nhead': 4,
 'nhid_factor': 2,
 'bptt': 10,
 'eval_positions': [9],
 'seq_len_used': 50,
 'sampling': 'mixed',
 'epochs': 400,
 'num_steps': 8192,
 'verbose': False,
 'mix_activations': True,
 'nan_prob_unknown_reason_reason_prior': 1.0,
 'categorical_feature_p': 0.2,
 'nan_prob_no_reason': 0.0,
 'nan_prob_unknown_reason': 0.0,
 'nan_prob_a_reason': 0.0,
 'max_num_classes': 10,
 'num_classes': 2,
 'noise_type': 'Gaussian',
 'balanced': False,
 'normalize_to_ranking': False,
 'set_value_to_nan': 0.1,
 'normalize_by_used_features': True,
 'num_features_used': <function tabpfn.scripts.model_builder.load_model.<locals>.<lambda>()>,
 'num_categorical_features_sampler_a': -1.0,
 'differentiable_hyperparameters': {'prior_bag_exp_weights_1': {'distribution': 'uniform',
   'min': 1000000.0,
   'max': 1000001.0},
  'num_layers': {'distribution': 'meta_trunc_norm_log_scaled',
   'max_mean': 6,

In [ ]:
model.model = model_pytorch

In [ ]:
import openml
import os
import numpy as np
# import train test split
from sklearn.model_selection import train_test_split
openml.config.cache_directory = os.path.expanduser('/storage/store/work/lgrinszt/openml_cache')
task_id = 361060
task = openml.tasks.get_task(task_id)
X, y = task.get_X_and_y()
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Restrict X_train to 1000 samples to speed up the example
X_train, y_train = X_train[:1000], y_train[:1000]
X_test, y_test = X_test[:5000], y_test[:5000]

model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
import openml
from sklearn.preprocessing import LabelEncoder
import pandas as pd
def balance_data(x, y):
    rng = np.random.RandomState(0)
    print("Balancing")
    print(x.shape)
    indices = [(y == i) for i in np.unique(y)]
    sorted_classes = np.argsort(
        list(map(sum, indices)))  # in case there are more than 2 classes, we take the two most numerous

    n_samples_min_class = sum(indices[sorted_classes[-2]])
    print("n_samples_min_class", n_samples_min_class)
    indices_max_class = rng.choice(np.where(indices[sorted_classes[-1]])[0], n_samples_min_class, replace=False)
    indices_min_class = np.where(indices[sorted_classes[-2]])[0]
    total_indices = np.concatenate((indices_max_class, indices_min_class))
    y = y[total_indices]
    indices_first_class = (y == sorted_classes[-1])
    indices_second_class = (y == sorted_classes[-2])
    y[indices_first_class] = 0
    y[indices_second_class] = 1

    return x.iloc[total_indices], y
def import_open_ml_data(dataset_id=None, task_id=None, remove_nans=None, impute_nans=None, categorical=False, regression=False, balance=False, rng=None) -> pd.DataFrame:
    """
    Import data from openML
    :param int openml_task_id:
    :param path_to_file:
    :return:
    """
    if task_id is not None:
        task = openml.tasks.get_task(task_id)  # download the OpenML task
        dataset = task.get_dataset()
    elif dataset_id is not None:
        dataset = openml.datasets.get_dataset(dataset_id)
    # retrieve categorical data for encoding
    X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute
    )
    categorical_indicator = np.array(categorical_indicator)
    print("{} categorical columns".format(sum(categorical_indicator)))
    print("{} columns".format(X.shape[1]))
    y_encoder = LabelEncoder()

    # Replace categorical values by integers for each categorical column
    for i, categorical in enumerate(categorical_indicator):
        X.iloc[:, i] = X.iloc[:, i].astype('category')
        X.iloc[:, i] = X.iloc[:, i].cat.codes
        X.iloc[:, i] = X.iloc[:, i].astype('int64')

    # remove missing values
    assert remove_nans or impute_nans, "You need to remove or impute nans"
    if remove_nans:
        missing_rows_mask = X.isnull().any(axis=1)
        if sum(missing_rows_mask) > X.shape[0] / 5:
            print("Removed {} rows with missing values on {} rows".format(
                sum(missing_rows_mask), X.shape[0]))
        X = X[~missing_rows_mask]
        y = y[~missing_rows_mask]
        n_rows_non_missing = X.shape[0]
        if n_rows_non_missing == 0:
            print("Removed all rows")
            return None
    elif impute_nans:
        from sklearn.impute import SimpleImputer
        # Impute numerical columns with mean and categorical columns with most frequent
        categorical_imputer = SimpleImputer(strategy="most_frequent")
        numerical_imputer = SimpleImputer(strategy="mean")
        # check that there a > 0 categorical columns
        if sum(categorical_indicator) > 0:
            X.iloc[:, categorical_indicator] = categorical_imputer.fit_transform(X.iloc[:, categorical_indicator])
        # check that there a > 0 numerical columns
        if sum(~categorical_indicator) > 0:
            X.iloc[:, ~categorical_indicator] = numerical_imputer.fit_transform(X.iloc[:, ~categorical_indicator])




    # print("removing {} categorical features among {} features".format(sum(categorical_indicator), X.shape[1]))
    # X = X.to_numpy()[:, ~categorical_indicator]  # remove all categorical columns
    # if X.shape[1] == 0:
    #     print("removed all features, skipping this task")
    #     return None

    y = y_encoder.fit_transform(y)



    if regression:
        y = y.astype(np.float64)
    else:
        y = y.astype(np.int64)

    if balance:
        X, y = balance_data(X, y)

    X = X.to_numpy()

    if categorical:
        return X, y, categorical_indicator

    return X, y, None

In [ ]:
X, y, _ = import_open_ml_data(task_id=361060, remove_nans=True, impute_nans=False, categorical=False, regression=False, balance=True, rng=None)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [ ]:
# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Restrict to 1000 samples for faster training
X_train, y_train = X_train[:1000], y_train[:1000]
X_test, y_test = X_test[:5000], y_test[:5000]

# Train a classifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)

# Score the classifier
print("Accuracy score Random Forest: {}".format(clf.score(X_test, y_test)))

# Train a classifier
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)

# Score the classifier
print("Accuracy score Gradient Boosting: {}".format(clf.score(X_test, y_test)))

# TabPFN
from tabpfn.scripts.transformer_prediction_interface import TabPFNClassifier

model = TabPFNClassifier()
model.fit(X_train, y_train)

# Score the classifier
print("Accuracy score TabPFN: {}".format(model.score(X_test, y_test)))

# Replace model in TabPFN
model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees25524_160.pt", 2, config_sample, 1)[0]
model.model = model_pytorch

# Fit
model.fit(X_train, y_train)

# Score the classifier
print("Accuracy score TabPFN trees: {}".format(model.score(X_test, y_test)))

# Replace model in TabPFN


model_pytorch = load_model("tabpfn/model_checkpoints", "model_mlp60400_160.pt", 2, config_sample, 1)[0]

model.model = model_pytorch

# Fit
model.fit(X_train, y_train)

# Score the classifier
print("Accuracy score TabPFN MLP: {}".format(model.score(X_test, y_test)))



model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees33888_8.pt", 2, config_sample, 1)[0]

model.model = model_pytorch

# Fit
model.fit(X_train, y_train)

# Score the classifier
print("Accuracy score TabPFN Trees 2: {}".format(model.score(X_test, y_test)))







In [ ]:
# Do the same thing for all tasks in openml suite 2
# and put everything in a pd dataframe
import pandas as pd
suite = openml.study.get_suite(337)
tasks = suite.tasks

n_repeat = 3

results = pd.DataFrame(columns=["task_id", "accuracy", "model"])
for task_id in tasks:
    print("Task id: {}".format(task_id))
    X, y, _ = import_open_ml_data(task_id=task_id, remove_nans=True, impute_nans=False, categorical=False, regression=False, balance=True, rng=None)
    if X.shape[1] > 100:
        print("skipping task {} because it has too many features".format(task_id))
        continue
    if X is None:
        print("skipping task {} because it has no features".format(task_id))
        continue

    # Evaluate the models n_repeat times

    for i in range(n_repeat):
        # Split the data into a training set and a test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)

        # Restrict to 1000 samples for faster training
        X_train, y_train = X_train[:1000], y_train[:1000]
        X_test, y_test = X_test[:5000], y_test[:5000]

        # Train a classifier
        clf = RandomForestClassifier()
        clf.fit(X_train, y_train)

        # Score the classifier
        accuracy = clf.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "Random Forest"}, ignore_index=True)

        # Train a classifier
        clf = GradientBoostingClassifier()
        clf.fit(X_train, y_train)

        # Score the classifier
        accuracy = clf.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "Gradient Boosting"}, ignore_index=True)

        # TabPFN
        model = TabPFNClassifier(N_ensemble_configurations=1)
        model.fit(X_train, y_train)

        # Score the classifier
        accuracy = model.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFNne"}, ignore_index=True)

        # TabPFN
        model = TabPFNClassifier()
        model.fit(X_train, y_train)

        # Score the classifier
        accuracy = model.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN"}, ignore_index=True)

        # Replace model in TabPFN
        model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees13368_60.pt", 2, config_sample, 1)[0]
        model.model = model_pytorch

        # Fit
        model.fit(X_train, y_train)

        # Score the classifier
        accuracy = model.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN trees"}, ignore_index=True)


        # Replace model in TabPFN
        model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees456_45.pt", 2, config_sample, 1)[0]
        model.model = model_pytorch

        # Fit
        model.fit(X_train, y_train)

        # Score the classifier
        accuracy = model.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN trees 2"}, ignore_index=True)



        # # Replace model in TabPFN
        # model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees60473_75.pt", 2, config_sample, 1)[0]
        # model.model = model_pytorch

        # # Fit
        # model.fit(X_train, y_train)

        # # Score the classifier
        # accuracy = model.score(X_test, y_test)
        # results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN trees 75"}, ignore_index=True)



        # # Replace model in TabPFN
        # model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees89438_60.pt", 2, config_sample, 1)[0]
        # model.model = model_pytorch

        # # Fit
        # model.fit(X_train, y_train)

        # # Score the classifier
        # accuracy = model.score(X_test, y_test)
        # results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN trees 2"}, ignore_index=True)

        # # Replace model in TabPFN
        # model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees89438_45.pt", 2, config_sample, 1)[0]
        # model.model = model_pytorch

        # # Fit
        # model.fit(X_train, y_train)

        # # Score the classifier
        # accuracy = model.score(X_test, y_test)
        # results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN trees 1"}, ignore_index=True)

        # # Replace model in TabPFN
        # model_pytorch = load_model("tabpfn/model_checkpoints", "model_mlp71353_400.pt", 2, config_sample, 1)[0]
        # model.model = model_pytorch

        # # Fit
        # model.fit(X_train, y_train)

        # # Score the classifier
        # accuracy = model.score(X_test, y_test)
        # results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN MLP"}, ignore_index=True)

results.to_csv("results.csv")    




In [ ]:
# Choose tasks with less than 100 features
accepted_tasks = []
for task_id in tasks:
    print("Task id: {}".format(task_id))
    X, y, _ = import_open_ml_data(task_id=task_id, remove_nans=True, impute_nans=False, categorical=False, regression=False, balance=True, rng=None)
    if X.shape[1] > 100:
        print("skipping task {} because it has too many features".format(task_id))
        continue
    if X is None:
        print("skipping task {} because it has no features".format(task_id))
        continue
    accepted_tasks.append(task_id)

results = results[results["task_id"].isin(accepted_tasks)]



In [ ]:
# Plot the results
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
fig = plt.figure(figsize=(20, 10))
# Convert tasks to string
results["task_id"] = results["task_id"].astype(str)
# scatter plot with jitter
sns.stripplot(x="task_id", y="accuracy", hue="model", data=results, jitter=0.05, dodge=True)
plt.show()


In [ ]:
2

In [ ]:
# Compute how results change for different checkpoints
# and put everything in a pd dataframe
import pandas as pd
suite = openml.study.get_suite(337)
tasks = suite.tasks

n_repeat = 3

results = pd.DataFrame(columns=["task_id", "accuracy", "model", "checkpoint"])
for task_id in tasks:
    print("Task id: {}".format(task_id))
    X, y, _ = import_open_ml_data(task_id=task_id, remove_nans=True, impute_nans=False, categorical=False, regression=False, balance=True, rng=None)
    if X.shape[1] > 100:
        print("skipping task {} because it has too many features".format(task_id))
        continue
    if X is None:
        print("skipping task {} because it has no features".format(task_id))
        continue

    # Evaluate the models n_repeat times

    for i in range(n_repeat):
        # Split the data into a training set and a test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)

        # Restrict to 1000 samples for faster training
        X_train, y_train = X_train[:1000], y_train[:1000]
        X_test, y_test = X_test[:5000], y_test[:5000]

        # TabPFN
        model = TabPFNClassifier()
        model.fit(X_train, y_train)

        # Score the classifier
        accuracy = model.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN", "checkpoint": np.nan}, ignore_index=True)

        for checkpoint in range(10, 400, 50):
            # Replace model in TabPFN
            model_pytorch = load_model("tabpfn/model_checkpoints", "model_mlp71353_{}.pt".format(checkpoint), 2, config_sample, 1)[0]
            model.model = model_pytorch

            # Fit
            model.fit(X_train, y_train)

            # Score the classifier
            accuracy = model.score(X_test, y_test)
            results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN MLP", "checkpoint": checkpoint}, ignore_index=True)
        
        # Use Gradient Boosting as a baseline
        clf = GradientBoostingClassifier()
        clf.fit(X_train, y_train)

        # Score the classifier
        accuracy = clf.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "Gradient Boosting", "checkpoint": np.nan}, ignore_index=True)
    

In [ ]:
# Plot the results
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
fig = plt.figure(figsize=(20, 10))
# Convert tasks to string
results["task_id"] = results["task_id"].astype(str)
# scatter plot with jitter
# Plot a different color for each checkpoint
ax = sns.stripplot(x="task_id", y="accuracy", hue="checkpoint", data=results, jitter=0.05, dodge=True)
# Plot the model without checkpoint
#ax2 = sns.stripplot(x="task_id", y="accuracy", hue="model", data=results[results["checkpoint"].isna()], jitter=0.05, dodge=True)
#ax = sns.stripplot(x="task_id", y="accuracy", hue="model", data=results, jitter=0.05, dodge=True)
plt.show()




In [ ]:
# Plot the results  for all  tasks and models in results.csv
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

results = pd.read_csv("results.csv")
results = results.drop(columns=["Unnamed: 0"])
results = results[results["accuracy"] < 1.0]
results = results[results["accuracy"] > 0.0]

# Plot the results for each task
# for task_id in results["task_id"].unique():
#     print("Task id: {}".format(task_id))
#     results_task = results[results["task_id"] == task_id]
#     ax = sns.boxplot(x="model", y="accuracy", data=results_task)
#     ax.set_title("Task id: {}".format(task_id))
#     plt.show()
# Replace by one scatterplot
ax = sns.scatterplot(x="model", y="accuracy", hue="task_id", data=results)



In [ ]:
print(2)

# Same for categorical data

In [22]:
# Do the same thing for all tasks in openml suite 2
# and put everything in a pd dataframe
import pandas as pd
suite = openml.study.get_suite(334) # categorical classification
tasks = suite.tasks

n_repeat = 3

results = pd.DataFrame(columns=["task_id", "accuracy", "model"])
for task_id in tasks:
    print("Task id: {}".format(task_id))
    X, y, _ = import_open_ml_data(task_id=task_id, remove_nans=True, impute_nans=False, categorical=True, regression=False, balance=True, rng=None)
    if X.shape[1] > 100:
        print("skipping task {} because it has too many features".format(task_id))
        continue
    if X is None:
        print("skipping task {} because it has no features".format(task_id))
        continue

    # Evaluate the models n_repeat times

    for i in range(n_repeat):
        # Split the data into a training set and a test set
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=i)

        # Restrict to 1000 samples for faster training
        X_train, y_train = X_train[:1000], y_train[:1000]
        X_test, y_test = X_test[:5000], y_test[:5000]

        # Train a classifier
        clf = RandomForestClassifier()
        clf.fit(X_train, y_train)

        # Score the classifier
        accuracy = clf.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "Random Forest"}, ignore_index=True)

        # Train a classifier
        clf = GradientBoostingClassifier()
        clf.fit(X_train, y_train)

        # Score the classifier
        accuracy = clf.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "Gradient Boosting"}, ignore_index=True)

        # TabPFN
        model = TabPFNClassifier(N_ensemble_configurations=1)
        model.fit(X_train, y_train)

        # Score the classifier
        accuracy = model.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFNne"}, ignore_index=True)

        # TabPFN
        model = TabPFNClassifier()
        model.fit(X_train, y_train)

        # Score the classifier
        accuracy = model.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN"}, ignore_index=True)
        # Replace model in TabPFN
        model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees13368_60.pt", 2, config_sample, 1)[0]
        model.model = model_pytorch

        # Fit
        model.fit(X_train, y_train)

        # Score the classifier
        accuracy = model.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN trees"}, ignore_index=True)


        # Replace model in TabPFN
        model_pytorch = load_model("tabpfn/model_checkpoints", "model_trees456_45.pt", 2, config_sample, 1)[0]
        model.model = model_pytorch

        # Fit
        model.fit(X_train, y_train)

        # Score the classifier
        accuracy = model.score(X_test, y_test)
        results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN trees 2"}, ignore_index=True)

results.to_csv("results_categorical.csv")    




Task id: 361110
1 categorical columns
8 columns
Balancing
(38474, 8)


/tmp/ipykernel_6963/406963134.py:48: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.iloc[:, i] = X.iloc[:, i].cat.codes
/tmp/ipykernel_6963/406963134.py:47: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.iloc[:, i] = X.iloc[:, i].astype('category')
/tmp/ipykernel_6963/406963134.py:48: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(

n_samples_min_class 19237


/tmp/ipykernel_6963/1826081905.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "Random Forest"}, ignore_index=True)
/tmp/ipykernel_6963/1826081905.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "Gradient Boosting"}, ignore_index=True)


Using style prior: True
{'recompute_attn': True}
<module 'wandb' (namespace)>
Using cpu device
Using a Transformer with 25.82 M parameters


...

/tmp/ipykernel_6963/1826081905.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN"}, ignore_index=True)


Using style prior: True
{'recompute_attn': True}
<module 'wandb' (namespace)>
Using 2 device
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_6963/1826081905.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN trees"}, ignore_index=True)


Using style prior: True
{'recompute_attn': True}
<module 'wandb' (namespace)>
Using 2 device
Using a Transformer with 25.82 M parameters


/tmp/ipykernel_6963/1826081905.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({"task_id": task_id, "accuracy": accuracy, "model": "TabPFN trees 2"}, ignore_index=True)


In [23]:
results

,task_id,accuracy,model
0,361110,0.786600,Random Forest
1,361110,0.782000,Gradient Boosting
2,361110,0.779000,TabPFNne
3,361110,0.776800,TabPFN
4,361110,0.769400,TabPFN trees
...,...,...,...
121,361286,0.635815,Gradient Boosting
122,361286,0.677062,TabPFNne
123,361286,0.678068,TabPFN
124,361286,0.660966,TabPFN trees


In [ ]:
# Choose tasks with less than 100 features
accepted_tasks = []
for task_id in tasks:
    print("Task id: {}".format(task_id))
    X, y, _ = import_open_ml_data(task_id=task_id, remove_nans=True, impute_nans=False, categorical=True, regression=False, balance=True, rng=None)
    if X.shape[1] > 100:
        print("skipping task {} because it has too many features".format(task_id))
        continue
    if X is None:
        print("skipping task {} because it has no features".format(task_id))
        continue
    accepted_tasks.append(task_id)

results = results[results["task_id"].isin(accepted_tasks)]



In [ ]:
# Plot the results
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")
fig = plt.figure(figsize=(20, 10))
# Convert tasks to string
results["task_id"] = results["task_id"].astype(str)
# scatter plot with jitter
sns.stripplot(x="task_id", y="accuracy", hue="model", data=results, jitter=0.05, dodge=True)
plt.show()


In [ ]:
results